In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip3 install catboost
!pip3 install category_encoders
!pip install -U scikit-learn
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.8/81.8 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 100.2 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.6 MB/s eta 0:00:00


In [1]:
import os
import datetime
import numpy as np
import pandas as pd
import unicodedata
from pathlib import Path
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    LabelEncoder, KBinsDiscretizer, RobustScaler, OneHotEncoder
)
from category_encoders import CountEncoder
import optuna
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.preprocessing import TargetEncoder
import warnings
warnings.simplefilter("ignore")

In [2]:
# Constants
exp_name = 'exp00047'
INPUT_DIR = Path("/content/drive/MyDrive/Colab Notebooks/signate2023/")
OUTPUT_PATH = f'/content/drive/MyDrive/Colab Notebooks/signate2023/{exp_name}/'
SUBMISSION_DIR = os.path.join(OUTPUT_PATH, 'submissions')
SUBMISSION_PATH = os.path.join(SUBMISSION_DIR, f'submission_{exp_name}.csv')
MODEL_DIR = os.path.join(OUTPUT_PATH, 'signate-models')
os.makedirs(MODEL_DIR, exist_ok=True)

# Load Data
train_df = pd.read_csv(INPUT_DIR / "train.csv")
test_df = pd.read_csv(INPUT_DIR / "test.csv")

In [3]:
state_dic = {}

# データ数的に、重複は気にせず上書きしていっても大した時間にはならない
for i in train_df.index:
    # ただし、欠損値で上書きされると困るので、そこだけケア
    if pd.isna(train_df.at[i, 'state'])==False:
        state_dic[train_df.at[i, 'region']] = train_df.at[i, 'state']

train_df.loc[pd.isna(train_df['state']).index, 'state'] = train_df.loc[pd.isna(train_df['state']).index, 'region'].map(state_dic)
test_df.loc[pd.isna(test_df['state']).index, 'state'] = test_df.loc[pd.isna(test_df['state']).index, 'region'].map(state_dic)

train_df.drop(['region'], axis=1, inplace=True)
test_df.drop(['region'], axis=1, inplace=True)

In [4]:
# Define a function to normalize manufacturer names
def normalize_manufacturer_name(name):
    # Convert full-width characters to half-width
    name = unicodedata.normalize('NFKC', name)
    # Convert to lowercase
    name = name.lower()
    # Remove non-ASCII characters
    name = ''.join(char for char in name if ord(char) < 128)
    return name


#fitしてtransformしないものはここで処理
def preprocessing(df):
    df[["year", "odometer"]] = df[["year", "odometer"]].astype("float32")

    df["cylinders"] = df["cylinders"].astype(str).str.extract("(\d+)").astype("float32")

    # sizeの表記揺れを修正
    df["size"] = df["size"].str.replace("ー", "-").astype(str)
    df["size"] = df["size"].str.replace("−", "-").astype(str)

    # Apply the function to the 'manufacturer' column
    df['manufacturer'] = df['manufacturer'].apply(normalize_manufacturer_name)

    df.loc[df['odometer'] == -1, 'odometer'] = np.nan


    # Define a dictionary for manufacturer name mapping with special characters
    manufacturer_name_mapping = {
        'nisan': 'nissan',
        'toyot': 'toyota',
        'nisѕan': 'nissan',
        'subαru': 'subaru',
        'subru': 'subaru',
        'sαturn': 'saturn',
        'lexu':'lexus',
        'аcura': 'acura',
        'vоlkswagen': 'volkswagen',
        'vlkswagen': 'volkswagen',
        'lexuѕ': 'lexus',
        'ᴄhrysler': 'chrysler',
        'sturn': 'saturn'
    }

    # Replace the manufacturer names using the mapping
    df['manufacturer'] = df['manufacturer'].replace(manufacturer_name_mapping)


    year_dict = {
        2999:1999,
        3008:2008,
        3011:2011,
        3015:2015,
        3017:2017,
        3019:2019,
    }
    df["year"] = df["year"].replace(year_dict)

    # 走行距離 / 製造年
    df["odometer_per_year"] = df["odometer"] / (2023 - df["year"])

    df['odometer_log'] = np.log1p(df['odometer'])

    # Create a new feature 'car_age'
    current_year = datetime.datetime.now().year
    df['car_age'] = current_year - df['year']

    # odometer_scaledとcar_ageの交互作用を新たな特徴量として作成します
    df['odometer_age_interaction'] = df['odometer'] * df['car_age']

    # 2. 'odometer'の時間経過による変化
    df['odometer_per_car_age'] = df['odometer'] / df['car_age']

    # 5. 'odometer'のビン化
    df['odometer_bin_100k'] = pd.cut(df['odometer'], bins=[0, 100000, 200000, 300000, np.inf], labels=False)

    # 6. 'odometer'の他の数学的変換
    df['odometer_sqrt'] = np.sqrt(df['odometer'])

    return df

In [5]:
train_feat_df = preprocessing(train_df)
test_feat_df = preprocessing(test_df)

In [6]:
pd.set_option('display.max_columns', None)
train_feat_df

,id,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state,price,odometer_per_year,odometer_log,car_age,odometer_age_interaction,odometer_per_car_age,odometer_bin_100k,odometer_sqrt
0,0,1949.0,bmw,excellent,6.0,gas,115148.0,clean,manual,rwd,mid-size,convertible,orange,tn,27587,1556.054077,11.653982,74.0,8520952.0,1556.054077,1.0,339.334656
1,1,2013.0,toyota,fair,8.0,gas,172038.0,clean,automatic,rwd,full-size,sedan,silver,pa,4724,17203.800781,12.055476,10.0,1720380.0,17203.800781,1.0,414.774628
2,2,1998.0,ford,good,6.0,gas,152492.0,clean,automatic,fwd,full-size,SUV,silver,ks,10931,6099.680176,11.934874,25.0,3812300.0,6099.680176,1.0,390.502228
3,3,2014.0,ford,excellent,4.0,gas,104118.0,clean,manual,fwd,mid-size,SUV,blue,ny,16553,11568.666992,11.553289,9.0,937062.0,11568.666992,1.0,322.673218
4,4,2005.0,ford,excellent,6.0,gas,144554.0,clean,manual,fwd,mid-size,sedan,red,ca,5158,8030.777832,11.881415,18.0,2601972.0,8030.777832,1.0,380.202576
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27527,27527,2008.0,ford,good,6.0,gas,26660.0,clean,automatic,rwd,compact,truck,black,pa,32212,1777.333374,10.190957,15.0,399900.0,1777.333374,0.0,163.278900
27528,27528,2007.0,ford,excellent,8.0,gas,108072.0,clean,automatic,rwd,full-size,pickup,black,ok,5400,6754.500000,11.590562,16.0,1729152.0,6754.500000,1.0,328.743073
27529,27529,2019.0,jeep,like new,6.0,gas,139908.0,clean,automatic,4wd,mid-size,SUV,white,ny,22227,34977.000000,11.848747,4.0,559632.0,34977.000000,1.0,374.042786
27530,27530,2007.0,jeep,excellent,6.0,gas,112326.0,clean,automatic,4wd,mid-size,sedan,white,ny,3054,7020.375000,11.629169,16.0,1797216.0,7020.375000,1.0,335.150726


In [7]:
def get_custom_cv(df, n_splits=5):
    # priceを小さい順に各foldに振り分ける
    df = df.sort_values(by="price", ignore_index=True)
    df["fold"] = [i for i in range(n_splits)] * (df.shape[0] // n_splits) \
                + [i for i in range(df.shape[0] % n_splits)]
    df = df.sort_values(by="id", ignore_index=True)


    for fold in range(n_splits):
        train_idx = df[df["fold"] != fold].index
        valid_idx = df[df["fold"] == fold].index
        yield train_idx, valid_idx

n_splits = 8
cv = list(get_custom_cv(train_feat_df, n_splits))

The variance of the mean of the folds:  4.695908800203918
0    0
1    3
2    0
3    2
4    3
5    3
6    3
7    3
8    0
9    4
Name: fold, dtype: int64
27522    4
27523    2
27524    2
27525    0
27526    0
27527    2
27528    1
27529    0
27530    1
27531    4
Name: fold, dtype: int64

In [8]:
train_feat_df

,id,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state,price,odometer_per_year,odometer_log,car_age,odometer_age_interaction,odometer_per_car_age,odometer_bin_100k,odometer_sqrt
0,0,1949.0,bmw,excellent,6.0,gas,115148.0,clean,manual,rwd,mid-size,convertible,orange,tn,27587,1556.054077,11.653982,74.0,8520952.0,1556.054077,1.0,339.334656
1,1,2013.0,toyota,fair,8.0,gas,172038.0,clean,automatic,rwd,full-size,sedan,silver,pa,4724,17203.800781,12.055476,10.0,1720380.0,17203.800781,1.0,414.774628
2,2,1998.0,ford,good,6.0,gas,152492.0,clean,automatic,fwd,full-size,SUV,silver,ks,10931,6099.680176,11.934874,25.0,3812300.0,6099.680176,1.0,390.502228
3,3,2014.0,ford,excellent,4.0,gas,104118.0,clean,manual,fwd,mid-size,SUV,blue,ny,16553,11568.666992,11.553289,9.0,937062.0,11568.666992,1.0,322.673218
4,4,2005.0,ford,excellent,6.0,gas,144554.0,clean,manual,fwd,mid-size,sedan,red,ca,5158,8030.777832,11.881415,18.0,2601972.0,8030.777832,1.0,380.202576
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27527,27527,2008.0,ford,good,6.0,gas,26660.0,clean,automatic,rwd,compact,truck,black,pa,32212,1777.333374,10.190957,15.0,399900.0,1777.333374,0.0,163.278900
27528,27528,2007.0,ford,excellent,8.0,gas,108072.0,clean,automatic,rwd,full-size,pickup,black,ok,5400,6754.500000,11.590562,16.0,1729152.0,6754.500000,1.0,328.743073
27529,27529,2019.0,jeep,like new,6.0,gas,139908.0,clean,automatic,4wd,mid-size,SUV,white,ny,22227,34977.000000,11.848747,4.0,559632.0,34977.000000,1.0,374.042786
27530,27530,2007.0,jeep,excellent,6.0,gas,112326.0,clean,automatic,4wd,mid-size,sedan,white,ny,3054,7020.375000,11.629169,16.0,1797216.0,7020.375000,1.0,335.150726


In [9]:
test_feat_df

,id,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state,odometer_per_year,odometer_log,car_age,odometer_age_interaction,odometer_per_car_age,odometer_bin_100k,odometer_sqrt
0,27532,2015.0,chevrolet,excellent,4.0,gas,92553.0,clean,automatic,fwd,full-size,SUV,red,co,11569.125000,11.435548,8.0,740424.0,11569.125000,0.0,304.225250
1,27533,2013.0,nissan,like new,4.0,gas,134385.0,salvage,automatic,fwd,mid-size,sedan,black,or,13438.500000,11.808472,10.0,1343850.0,13438.500000,1.0,366.585602
2,27534,2011.0,volkswagen,good,4.0,gas,102489.0,clean,automatic,fwd,full-size,sedan,black,oh,8540.750000,11.537520,12.0,1229868.0,8540.750000,1.0,320.139038
3,27535,2016.0,jeep,excellent,6.0,diesel,64310.0,clean,automatic,4wd,mid-size,SUV,red,co,9187.142578,11.071486,7.0,450170.0,9187.142578,0.0,253.594162
4,27536,1999.0,honda,excellent,8.0,gas,180839.0,rebuilt,automatic,4wd,mid-size,SUV,silver,nc,7534.958496,12.105368,24.0,4340136.0,7534.958496,1.0,425.251678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27532,55064,2016.0,gmc,excellent,6.0,gas,90902.0,rebuilt,automatic,fwd,full-size,truck,black,mt,12986.000000,11.417548,7.0,636314.0,12986.000000,0.0,301.499573
27533,55065,2012.0,nissan,excellent,4.0,gas,27234.0,rebuilt,automatic,fwd,mid-size,sedan,white,nv,2475.818115,10.212258,11.0,299574.0,2475.818115,0.0,165.027267
27534,55066,2002.0,bmw,excellent,6.0,gas,99761.0,clean,automatic,rwd,full-size,coupe,blue,az,4750.523926,11.510543,21.0,2094981.0,4750.523926,0.0,315.849640
27535,55067,2006.0,bmw,excellent,6.0,gas,162279.0,clean,automatic,4wd,mid-size,sedan,silver,co,9545.823242,11.997079,17.0,2758743.0,9545.823242,1.0,402.838684


In [10]:
def preprocessing_per_fold(X_tr, y_tr, X_val, transformed_test_data, te_cols):
    scaler = RobustScaler()
    ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
    imputer = SimpleImputer(strategy='median')

    # RobustScaler
    X_tr['odometer_scaled'] = scaler.fit_transform(X_tr[['odometer']])
    X_val['odometer_scaled'] = scaler.transform(X_val[['odometer']])
    transformed_test_data['odometer_scaled'] = scaler.transform(transformed_test_data[['odometer']])

    X_tr["odometer_map_30"], bins = pd.cut(X_tr["odometer"], bins=30, labels=False, retbins=True)
    X_val["odometer_map_30"] = pd.cut(X_val["odometer"], bins=bins, labels=False)
    transformed_test_data["odometer_map_30"] = pd.cut(transformed_test_data["odometer"], bins=bins, labels=False)
    X_tr["odometer_map_40"], bins = pd.cut(X_tr["odometer"], bins=40, labels=False, retbins=True)
    X_val["odometer_map_40"] = pd.cut(X_val["odometer"], bins=bins, labels=False)
    transformed_test_data["odometer_map_40"] = pd.cut(transformed_test_data["odometer"], bins=bins, labels=False)


    # Binning
    X_tr['odometer_scaled_bin'] = pd.cut(X_tr['odometer'], bins=10, labels=False)
    X_val['odometer_scaled_bin'] = pd.cut(X_val['odometer'], bins=10, labels=False)
    transformed_test_data['odometer_scaled_bin'] = pd.cut(transformed_test_data['odometer'], bins=10, labels=False)

    X_tr['car_age_bin'] = pd.cut(X_tr['car_age'], bins=10, labels=False)
    X_val['car_age_bin'] = pd.cut(X_val['car_age'], bins=10, labels=False)
    transformed_test_data['car_age_bin'] = pd.cut(transformed_test_data['car_age'], bins=10, labels=False)



    # One-Hot Encoding for categorical columns
    onehot_cols = ['manufacturer', 'condition', 'fuel', 'title_status',
            'transmission', 'drive', 'size', 'type', 'paint_color', 'state']
    ohe.fit(X_tr[onehot_cols])
    columns_ohe = ohe.get_feature_names_out(onehot_cols)

    # Transform train, valid and test data
    X_ohe_tr = ohe.transform(X_tr[onehot_cols])
    X_ohe_val = ohe.transform(X_val[onehot_cols])
    test_ohe = ohe.transform(transformed_test_data[onehot_cols])

    # Add the OHE columns to the dataframes
    X_tr = pd.concat([X_tr, pd.DataFrame(X_ohe_tr, columns=columns_ohe, index=X_tr.index)], axis=1)
    X_val = pd.concat([X_val, pd.DataFrame(X_ohe_val, columns=columns_ohe, index=X_val.index)], axis=1)
    transformed_test_data = pd.concat([transformed_test_data, pd.DataFrame(test_ohe, columns=columns_ohe, index=transformed_test_data.index)], axis=1)


    #クラスタリング
    # SimpleImputerをインスタンス化（ここでは欠損値を中央値で補完）
    imputer = SimpleImputer(strategy='median')
    # 補完を行いたい特徴量を選択
    features = X_tr[['odometer', 'car_age']].copy()
    # 欠損値補完
    features_imputed = imputer.fit_transform(features)
    # KMeansクラスタリングを設定
    kmeans = KMeans(n_clusters=10, random_state=0)
    # 補完されたデータに基づいてクラスタリング
    X_tr['odometer_age_cluster'] = kmeans.fit_predict(features_imputed)
    # 検証データについても同様に処理
    features_val = X_val[['odometer', 'car_age']].copy()
    features_val_imputed = imputer.transform(features_val)
    X_val['odometer_age_cluster'] = kmeans.predict(features_val_imputed)
    # テストデータについても同様に処理
    features_test = transformed_test_data[['odometer', 'car_age']].copy()
    features_test_imputed = imputer.transform(features_test)
    transformed_test_data['odometer_age_cluster'] = kmeans.predict(features_test_imputed)
    # 元のodometer_scaledの欠損値を復元
    X_tr['odometer'] = features['odometer']
    X_val['odometer'] = features_val['odometer']
    transformed_test_data['odometer'] = features_test['odometer']


    # 1. odometerに対する他の特徴量の比率
    mean_odometer_per_year = X_tr.groupby('year')['odometer'].transform('mean')
    X_tr['odometer_ratio_year'] = X_tr['odometer'] / mean_odometer_per_year
    X_val['odometer_ratio_year'] = X_val['odometer'] / mean_odometer_per_year
    transformed_test_data['odometer_ratio_year'] = transformed_test_data['odometer'] / mean_odometer_per_year

    # Count encoding
    count_cols = ["cylinders", "manufacturer", "condition", "fuel", "title_status", "transmission", "drive", "size", "type", "paint_color", "state",]
    for col in count_cols:
        count_map = X_tr[col].value_counts().to_dict()
        X_tr[col + '_count'] = X_tr[col].map(count_map)
        X_val[col + '_count'] = X_val[col].map(count_map)
        transformed_test_data[col + '_count'] = transformed_test_data[col].map(count_map)

    # Aggregate features
    agg_cols = ["manufacturer", "cylinders", "condition", "fuel", "drive", "size", "type"]

    for col in agg_cols:
        for agg_func in ['mean', 'std']:
            agg_map = X_tr.groupby(col)['odometer'].agg(agg_func).to_dict()
            X_tr[col + '_odometer_' + agg_func] = X_tr[col].map(agg_map)
            X_val[col + '_odometer_' + agg_func] = X_val[col].map(agg_map)
            transformed_test_data[col + '_odometer_' + agg_func] = transformed_test_data[col].map(agg_map)

    # Target encoding
    for col in te_cols:
        te = TargetEncoder(target_type="continuous", random_state=88)
        te.fit(X_tr[[col]], y_tr)
        X_tr['te_' + col] = te.transform(X_tr[[col]])
        X_val['te_' + col] = te.transform(X_val[[col]])
        transformed_test_data['te_' + col] = te.transform(transformed_test_data[[col]])


    #2変数ターゲットエンコーディング
    X_tr["year_map"], bins = pd.cut(X_tr["year"], bins=20, labels=False, retbins=True)
    X_val["year_map"] = pd.cut(X_val["year"], bins=bins, labels=False)
    transformed_test_data["year_map"] = pd.cut(transformed_test_data["year"], bins=bins, labels=False)
    X_tr["odometer_map"], bins = pd.cut(X_tr["odometer"], bins=20, labels=False, retbins=True)
    X_val["odometer_map"] = pd.cut(X_val["odometer"], bins=bins, labels=False)
    transformed_test_data["odometer_map"] = pd.cut(transformed_test_data["odometer"], bins=bins, labels=False)


    cross_features = [
        'year_map', 'manufacturer', 'condition', 'cylinders','fuel', 'odometer_map', 'title_status', 'transmission', 'drive', 'size',
        'type', 'paint_color', 'state', 'odometer_age_cluster'
    ]

    for i, col1 in enumerate(cross_features):
        for col2 in cross_features[i+1:]:
            tmp = X_tr.groupby([col1, col2])["price"].mean().reset_index()
            X_tr = pd.merge(X_tr, tmp.rename(columns={"price":f"{col1}*{col2}_price"}), on=[col1, col2], how="left")
            X_val = pd.merge(X_val, tmp.rename(columns={"price":f"{col1}*{col2}_price"}), on=[col1, col2], how="left")
            transformed_test_data = pd.merge(transformed_test_data, tmp.rename(columns={"price":f"{col1}*{col2}_price"}), on=[col1, col2], how="left")




    return X_tr, X_val, transformed_test_data


In [11]:
def fit_lgbm(X, y, test_data, cv, te_cols, categorical_cols: list = None, params: dict = None, verbose: int = 50):
    if params is None:
        params = {}

    models = []
    target_encoders = []
    transformed_test_data_list = []
    n_records = len(X)
    oof_pred = np.zeros((n_records,), dtype=np.float32)

    for i, (idx_train, idx_valid) in enumerate(cv):
        X_tr, y_tr = X.iloc[idx_train], y.iloc[idx_train]
        X_val, y_val = X.iloc[idx_valid], y.iloc[idx_valid]
        transformed_test_data = test_data.copy()

        X_tr, X_val, transformed_test_data = preprocessing_per_fold(X_tr, y_tr, X_val, transformed_test_data, te_cols)

        drop_cols = ['id', 'manufacturer', 'condition', 'fuel', 'title_status',
                     'transmission', 'drive', 'size', 'type', 'paint_color', 'state']
        X_tr.drop(drop_cols+['price'], axis=1, inplace=True)
        X_val.drop(drop_cols+['price'], axis=1, inplace=True)
        transformed_test_data.drop(drop_cols, axis=1, inplace=True)


        transformed_test_data_list.append(transformed_test_data)


        model = lgb.LGBMRegressor(**params)
        model.fit(
            X_tr,
            y_tr,
            categorical_feature=categorical_cols,
            eval_set=[(X_val, y_val)],
            callbacks=[lgb.early_stopping(100, verbose=verbose)],
        )
        pred_i = model.predict(X_val)
        oof_pred[idx_valid] = pred_i
        models.append(model)
        score = mean_absolute_percentage_error(y_val, pred_i)
        print(f" - fold{i + 1} - {score:.4f}")

    score = mean_absolute_percentage_error(y, oof_pred)
    print("=" * 50)
    print(f"FINISHI: Whole Score: {score:.4f}")
    return score, oof_pred, models, transformed_test_data_list




# optunaを使ったパラメータチューニング
def tuning(train_feat_df, train_df, cv):
     def objective(trial):
         max_depth = trial.suggest_int("max_depth", 1, 10)
         params = {
              "max_depth": max_depth,
              "n_estimators": trial.suggest_int('n_estimators', 100, 50000),
              "learning_rate": trial.suggest_float("learning_rate", 0.01, 1.0),
              "num_leaves": trial.suggest_int("num_leaves", 2, 2**max_depth),
              "colsample_bytree": trial.suggest_uniform("colsample_bytree", 0.1, 1.0),
              "subsample": trial.suggest_float("subsample", 0.01, 1.0),
              "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 0, 300),
              "max_bin": trial.suggest_int("max_bin", 32, 512),
              "reg_lambda": trial.suggest_float("reg_lambda", 1e-10, 100.0),
              "task":"train",
              "objective": "mape",
              "boosting":"gbdt",
              "metric": 'mape',
              "importance_type": "gain",
              "seed" : 88,
              "verbosity": -1,


         }

         te_cols = [
            "cylinders",
            "manufacturer",
            "condition",
            "fuel",
            "title_status",
            "transmission",
            "drive",
            "size",
            "type",
            "paint_color",
            "state",
            ]

         score, _, _, _ = fit_lgbm(
             X=train_feat_df, y=train_df["price"], test_data=test_feat_df, cv=cv, te_cols=te_cols,  categorical_cols=[], params=params, verbose=-1
         )
         return score

     study = optuna.create_study(direction="minimize")
     study.optimize(objective, n_trials=100)
     print("Number of finished trials:", len(study.trials))
     print("Best trial:", study.best_trial.params)
     return study.best_trial.params

In [18]:
lgbm_params = {
    "task":"train",
    "objective": "mape",
    "boosting":"gbdt",
    "metric": 'mape',
    "seed" : 88,
    "importance_type": "gain",
    "random_state": 88,
    "verbose": -1,
    "n_estimators": 15683,
    "learning_rate": 0.03176950913176005,
    "max_depth": 7,
    "num_leaves": 64,
    "colsample_bytree": 0.17075595640937685,
    "subsample":  0.19095496529419081,

}

te_cols = [
            "cylinders",
            "manufacturer",
            "condition",
            "fuel",
            "title_status",
            "transmission",
            "drive",
            "size",
            "type",
            "paint_color",
            "state",
            ]

# training
score, oof_lgb, models_lgb, transformed_test_data_list  = fit_lgbm(
    train_feat_df,
    y=train_df["price"],
    test_data=test_feat_df,
    te_cols=te_cols,
    categorical_cols=[],
    params=lgbm_params,
    cv=cv,
    verbose=False,
)

 - fold1 - 0.4406
 - fold2 - 0.4503
 - fold3 - 0.4378
 - fold4 - 0.4544
 - fold5 - 0.4365
 - fold6 - 0.4364
 - fold7 - 0.4497
 - fold8 - 0.4429
FINISHI: Whole Score: 0.4436


In [25]:


car_age_binのワンホットエンコーディングとった
 - fold1 - 0.4400
 - fold2 - 0.4507
 - fold3 - 0.4383
 - fold4 - 0.4543
 - fold5 - 0.4360
 - fold6 - 0.4366
 - fold7 - 0.4509
 - fold8 - 0.4416
==================================================
FINISHI: Whole Score: 0.4435




odometer_scaled_binのワンホットエンコーディングとった
- fold1 - 0.4389
 - fold2 - 0.4500
 - fold3 - 0.4386
 - fold4 - 0.4545
 - fold5 - 0.4356
 - fold6 - 0.4364
 - fold7 - 0.4510
 - fold8 - 0.4422
==================================================
FINISHI: Whole Score: 0.4434


 odometer_map_30のワンホットエンコーディングなし
 - fold1 - 0.4401
 - fold2 - 0.4505
 - fold3 - 0.4388
 - fold4 - 0.4536
 - fold5 - 0.4359
 - fold6 - 0.4363
 - fold7 - 0.4508
 - fold8 - 0.4432
==================================================
FINISHI: Whole Score: 0.4436

 odometer_map_30のワンホットエンコーディングあり
 - fold1 - 0.4394
 - fold2 - 0.4516
 - fold3 - 0.4395
 - fold4 - 0.4544
 - fold5 - 0.4370
 - fold6 - 0.4357
 - fold7 - 0.4516
 - fold8 - 0.4423
==================================================
FINISHI: Whole Score: 0.4439

IndentationError: ignored

In [ ]:
pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp00046/signate-models/X_tr.csv')

,id,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state,price,odometer_per_year,odometer_log,car_age,odometer_age_interaction,odometer_per_car_age,odometer_bin_100k,odometer_sqrt,odometer_scaled,odometer_scaled_bin_0.0,odometer_scaled_bin_1.0,odometer_scaled_bin_4.0,odometer_scaled_bin_5.0,odometer_scaled_bin_9.0,odometer_scaled_bin_nan,car_age_bin_0,car_age_bin_1,car_age_bin_2,car_age_bin_3,car_age_bin_4,car_age_bin_5,car_age_bin_6,car_age_bin_7,car_age_bin_8,car_age_bin_9,manufacturer_acura,manufacturer_alfa-romeo,manufacturer_aston-martin,manufacturer_audi,manufacturer_bmw,manufacturer_buick,manufacturer_cadillac,manufacturer_chevrolet,manufacturer_chrysler,manufacturer_cura,manufacturer_dodge,manufacturer_fiat,manufacturer_ford,manufacturer_gmc,manufacturer_honda,manufacturer_hyundai,manufacturer_infiniti,manufacturer_jaguar,manufacturer_jeep,manufacturer_kia,manufacturer_land rover,manufacturer_lexus,manufacturer_lincoln,manufacturer_mazda,manufacturer_mercedes-benz,manufacturer_mercury,manufacturer_mini,manufacturer_mitsubishi,manufacturer_nissan,manufacturer_pontiac,manufacturer_porsche,manufacturer_ram,manufacturer_rover,manufacturer_saturn,manufacturer_subaru,manufacturer_toyota,manufacturer_volkswagen,manufacturer_volvo,condition_excellent,condition_fair,condition_good,condition_like new,condition_new,condition_salvage,fuel_diesel,fuel_electric,fuel_gas,fuel_hybrid,fuel_other,fuel_nan,title_status_clean,title_status_lien,title_status_missing,title_status_parts only,title_status_rebuilt,title_status_salvage,title_status_nan,transmission_automatic,transmission_manual,transmission_other,drive_4wd,drive_fwd,drive_rwd,size_compact,size_full-size,size_mid-size,size_sub-compact,type_SUV,type_bus,type_convertible,type_coupe,type_hatchback,type_mini-van,type_offroad,type_other,type_pickup,type_sedan,type_truck,type_van,type_wagon,type_nan,paint_color_black,paint_color_blue,paint_color_brown,paint_color_custom,paint_color_green,paint_color_grey,paint_color_orange,paint_color_purple,paint_color_red,paint_color_silver,paint_color_white,paint_color_yellow,state_ak,state_al,state_ar,state_az,state_ca,state_co,state_ct,state_dc,state_de,state_fl,state_ga,state_hi,state_ia,state_id,state_il,state_in,state_ks,state_ky,state_la,state_ma,state_md,state_me,state_mi,state_mn,state_mo,state_ms,state_mt,state_nc,state_nd,state_ne,state_nh,state_nj,state_nm,state_nv,state_ny,state_oh,state_ok,state_or,state_pa,state_ri,state_sc,state_sd,state_tn,state_tx,state_ut,state_va,state_vt,state_wa,state_wi,state_wv,state_wy,state_nan,odometer_age_cluster,odometer_ratio_year,cylinders_count,manufacturer_count,condition_count,fuel_count,title_status_count,transmission_count,drive_count,size_count,type_count,paint_color_count,state_count,manufacturer_odometer_mean,manufacturer_odometer_std,cylinders_odometer_mean,cylinders_odometer_std,condition_odometer_mean,condition_odometer_std,fuel_odometer_mean,fuel_odometer_std,drive_odometer_mean,drive_odometer_std,size_odometer_mean,size_odometer_std,type_odometer_mean,type_odometer_std,te_cylinders,te_manufacturer,te_condition,te_fuel,te_title_status,te_transmission,te_drive,te_size,te_type,te_paint_color,te_state
0,0,1949.0,bmw,excellent,6.0,gas,115148.0,clean,manual,rwd,mid-size,convertible,orange,tn,27587,1556.0541,11.653982,74.0,8520952.0,1556.0541,1.0,339.33466,0.046804,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [12]:
lgbm_params = tuning(train_feat_df, train_df, cv)

lgbm_params.update(
    {
    "task":"train",
    "objective": "mape",
    "boosting":"gbdt",
    "metric": 'mape',
    "seed" : 88,
    "verbosity": -1,
    }
)

te_cols = [
            "cylinders",
            "manufacturer",
            "condition",
            "fuel",
            "title_status",
            "transmission",
            "drive",
            "size",
            "type",
            "paint_color",
            "state",
            ]

# training
score, oof_lgb, models_lgb, transformed_test_data_list = fit_lgbm(
    train_feat_df,
    y=train_df["price"],
    test_data=test_feat_df,
    te_cols=te_cols,
    categorical_cols=[],
    params=lgbm_params,
    cv=cv,
    verbose=False,
)

[I 2023-08-19 04:12:12,221] A new study created in memory with name: no-name-96f4cd96-5dc5-45ed-9684-3a45f88fab94


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[115]	valid_0's mape: 0.450107
 - fold1 - 0.4501
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[93]	valid_0's mape: 0.466429
 - fold2 - 0.4664
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[114]	valid_0's mape: 0.44757
 - fold3 - 0.4476
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[95]	valid_0's mape: 0.464656
 - fold4 - 0.4647
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[68]	valid_0's mape: 0.45067
 - fold5 - 0.4507
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[154]	valid_0's mape: 0.44218
 - fold6 - 0.4422
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[122]	valid_0's mape: 0.461318
 -

[I 2023-08-19 04:13:51,445] Trial 0 finished with value: 0.45459071593323364 and parameters: {'max_depth': 2, 'n_estimators': 15853, 'learning_rate': 0.772496790938999, 'num_leaves': 3, 'colsample_bytree': 0.9078626111403667, 'subsample': 0.9943038744501753, 'min_data_in_leaf': 268, 'max_bin': 113, 'reg_lambda': 85.14248078411103}. Best is trial 0 with value: 0.45459071593323364.


Early stopping, best iteration is:
[128]	valid_0's mape: 0.453793
 - fold8 - 0.4538
FINISHI: Whole Score: 0.4546
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[203]	valid_0's mape: 0.445339
 - fold1 - 0.4453
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[544]	valid_0's mape: 0.461176
 - fold2 - 0.4612
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[197]	valid_0's mape: 0.444291
 - fold3 - 0.4443
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[137]	valid_0's mape: 0.463048
 - fold4 - 0.4630
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[311]	valid_0's mape: 0.445509
 - fold5 - 0.4455
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[331]	valid_0's mape: 0.443843
 - fold6 - 0.4438
Training un

[I 2023-08-19 04:15:23,638] Trial 1 finished with value: 0.450987940676914 and parameters: {'max_depth': 2, 'n_estimators': 32016, 'learning_rate': 0.47236000501134157, 'num_leaves': 3, 'colsample_bytree': 0.436941097241786, 'subsample': 0.8571586951578418, 'min_data_in_leaf': 62, 'max_bin': 309, 'reg_lambda': 93.57356521499447}. Best is trial 1 with value: 0.450987940676914.


...

Early stopping, best iteration is:
[100]	valid_0's mape: 0.442172
 - fold8 - 0.4422
FINISHI: Whole Score: 0.4422
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[103]	valid_0's mape: 0.438595
 - fold1 - 0.4386
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[227]	valid_0's mape: 0.44804
 - fold2 - 0.4480
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[74]	valid_0's mape: 0.436917
 - fold3 - 0.4369
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[37]	valid_0's mape: 0.453699
 - fold4 - 0.4537
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[47]	valid_0's mape: 0.438022
 - fold5 - 0.4380
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[59]	valid_0's mape: 0.434309
 - fold6 - 0.4343
Training until v

[I 2023-08-19 06:30:34,105] Trial 98 finished with value: 0.4430465567177254 and parameters: {'max_depth': 8, 'n_estimators': 21522, 'learning_rate': 0.12097075456064293, 'num_leaves': 151, 'colsample_bytree': 0.10202803956900057, 'subsample': 0.3724890337518739, 'min_data_in_leaf': 154, 'max_bin': 40, 'reg_lambda': 50.89780479519137}. Best is trial 71 with value: 0.4406568970841084.


Early stopping, best iteration is:
[99]	valid_0's mape: 0.443282
 - fold8 - 0.4433
FINISHI: Whole Score: 0.4430
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[273]	valid_0's mape: 0.438787
 - fold1 - 0.4388
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[287]	valid_0's mape: 0.448627
 - fold2 - 0.4486
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[386]	valid_0's mape: 0.434698
 - fold3 - 0.4347
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[219]	valid_0's mape: 0.451347
 - fold4 - 0.4513
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[266]	valid_0's mape: 0.434925
 - fold5 - 0.4349
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[437]	valid_0's mape: 0.431467
 - fold6 - 0.4315
Training unt

[I 2023-08-19 06:31:58,610] Trial 99 finished with value: 0.4412544442101769 and parameters: {'max_depth': 6, 'n_estimators': 25502, 'learning_rate': 0.03020221177740447, 'num_leaves': 63, 'colsample_bytree': 0.17328045913510295, 'subsample': 0.32418906716548107, 'min_data_in_leaf': 157, 'max_bin': 32, 'reg_lambda': 53.169338203938324}. Best is trial 71 with value: 0.4406568970841084.


Early stopping, best iteration is:
[218]	valid_0's mape: 0.439959
 - fold8 - 0.4400
FINISHI: Whole Score: 0.4413
Number of finished trials: 100
Best trial: {'max_depth': 8, 'n_estimators': 15662, 'learning_rate': 0.011719450430539596, 'num_leaves': 122, 'colsample_bytree': 0.15321633494439757, 'subsample': 0.3961263473804087, 'min_data_in_leaf': 116, 'max_bin': 69, 'reg_lambda': 56.34682871444761}
 - fold1 - 0.4369
 - fold2 - 0.4469
 - fold3 - 0.4357
 - fold4 - 0.4517
 - fold5 - 0.4342
 - fold6 - 0.4320
 - fold7 - 0.4490
 - fold8 - 0.4389
FINISHI: Whole Score: 0.4407


In [ ]:
Best trial: {'max_depth': 8, 'n_estimators': 15662, 'learning_rate': 0.011719450430539596, 'num_leaves': 122, 'colsample_bytree': 0.15321633494439757, 'subsample': 0.3961263473804087, 'min_data_in_leaf': 116, 'max_bin': 69, 'reg_lambda': 56.34682871444761}
 - fold1 - 0.4369
 - fold2 - 0.4469
 - fold3 - 0.4357
 - fold4 - 0.4517
 - fold5 - 0.4342
 - fold6 - 0.4320
 - fold7 - 0.4490
 - fold8 - 0.4389
==================================================
FINISHI: Whole Score: 0.4407

In [ ]:
# Create a copy of the dataframe
#train_data_transformed = train_feat_df.copy()#

# Apply target encoding to the copy of the dataframe
#for col in te_cols:
#    # Create a new instance of TargetEncoder and fit it to the whole data
#    te = TargetEncoder(target_type="continuous", random_state=88)
#    te.fit(train_data_transformed[[col]], train_df["price"])
#    train_data_transformed['te_' + col] = te.transform(train_data_transformed[[col]])

## Drop the original columns
#drop_cols = ['ori__manufacturer', 'ori__condition', 'ori__fuel', 'ori__title_status',
#       'ori__transmission', 'ori__drive', 'ori__size', 'ori__type', 'ori__paint_color', 'ori__state']
#train_data_transformed.drop(drop_cols, axis=1, inplace=True)

## Check the feature importances
#importances = pd.DataFrame({
#    'Feature': train_data_transformed.columns,
##    'Importance': models_lgb[0].feature_importances_
#}).sort_values(by='Importance', ascending=False)
#importances


In [13]:
sub_df = pd.read_csv(INPUT_DIR / "submit_sample.csv", names=["id", "price"])

In [14]:
preds = []
for transformed_test_data, model in zip(transformed_test_data_list, models_lgb):
    pred = model.predict(transformed_test_data)
    preds.append(pred)
pred = np.mean(preds, axis=0)

# submission
sub_df["price"] = pred
sub_df.to_csv(f'{MODEL_DIR}/submission.csv', index=False, header=False)


In [15]:
pd.concat([train_df['id'], pd.DataFrame(oof_lgb, columns=['pred'])]).to_csv(f'{MODEL_DIR}/exp47_oof_pred.csv', index=False)